# Day 23: Unstable Diffusion

[*Advent of Code 2022 day 23*](https://adventofcode.com/2022/day/23) and [*solution megathread*](https://redd.it/zt6xz5)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2022/23/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2022%2F23%2Fcode.ipynb)

In [1]:
import sys
sys.path.append('../../')


%load_ext nb_mypy
%nb_mypy On

Version 1.0.4


In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

%load_ext pycodestyle_magic
%pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [3]:
from IPython.display import HTML

HTML(downloaded['part1'])

## Comments

...

In [4]:
testdata = """....#..
..###.#
#...#.#
.#...##
#.###..
##.#.##
.#..#..""".splitlines()

testdata2 = """.....
..##.
..#..
.....
..##.
.....""".splitlines()

inputdata = downloaded['input'].splitlines()
# inputdata = open('input.txt', 'r').read().splitlines()

In [5]:
from IPython.display import display

display(f'{inputdata[:10]} ... {len(inputdata)=}')

"['.##.#.#......###..######...##.##.#..#.#.#.#######.###.#####...#...#..#', '#...######.#.#.#.##....#..##.###.#..##.#..#.....##.....#....###.##.##.', '.##.#.##..#.....#...###..#...##.##...........#.#..###.###...####.##..#', '##.#.##.#.####......#####..#.....#..#....###..#..####.##.#.###....####', '.....#...#.######...###..#....##.#.####.###.#..###...#.#.#..###.##...#', '#.#...#.##.....#..#######..##.###.###.####.#.##..##........#.#####..#.', '.#..###..#.....#..##.....#..#.##....#....##..###..#####.#....##.#.#..#', '#..##.##.#.##.#..#.#.#.#..#.#.#..#.#...######.###.#..##.##...##.#.##.#', '.##....#....#..#..#.###.......##.#..###..#....#.####....##.#.##...###.', '.....#.###.#....##.....#...###...##.#.#.#.....###.#..#..##..#..##.#.##'] ... len(inputdata)=70"

In [6]:
from typing import Tuple, List, Set, Iterable, Callable

Coord = Tuple[int, ...]


def parse_data(data: List[str]):
    elves: Set[Coord] = set()
    for row, line in enumerate(data):
        for column, character in enumerate(line):
            if character == '#':
                elves.add((row, column))
    return elves


def window(elves: Set[Coord]) -> Tuple[Coord, Coord]:
    def pick(criteria: Callable[[Iterable[int]], int]) -> Coord:
        return tuple(criteria(elf[i] for elf in elves) for i in (0, 1))
    return pick(min), pick(max)


def elves_str(elves: Set[Coord]) -> List[str]:
    def render(tile: Coord) -> str:
        if tile in elves:
            return '#'
        else:
            return '.'
    output: List[str] = list()
    mins, maxs = window(elves)
    for row in range(mins[0] - 1, maxs[0] + 2):
        output.append(''.join(render((row, col))
                              for col in range(mins[1] - 1, maxs[1] + 1)))
    return output


def count_spaces(elves: Set[Coord]) -> int:
    mins, maxs = window(elves)
    return (maxs[0] - mins[0] + 1)*(maxs[1] - mins[1] + 1) - len(elves)

In [7]:
from typing import Dict

HEADINGS: Dict[str, Coord] = {
    'NW': (-1, -1), 'N': (-1, 0), 'NE': (-1, 1),
    'W': (0, -1), 'E': (0, 1),
    'SW': (1, -1), 'S': (1, 0), 'SE': (1, 1)}

PREFERENCES: List[Tuple[Tuple[str, ...], Coord]] = [
    (('N', 'NE', 'NW'), HEADINGS['N']),
    (('S', 'SE', 'SW'), HEADINGS['S']),
    (('W', 'NW', 'SW'), HEADINGS['W']),
    (('E', 'NE', 'SE'), HEADINGS['E'])]

In [8]:
def add_tuples(a: Coord, b: Coord) -> Coord:
    assert len(a) == len(b)
    return tuple(ai + bi for ai, bi in zip(a, b))


def neighbours(elf: Coord, elves: Set[Coord]) -> Dict[str, Coord]:
    output: Dict[str, Coord] = dict()
    for heading, offset in HEADINGS.items():
        neighbour = add_tuples(elf, offset)
        if neighbour in elves:
            output[heading] = neighbour
    return output

In [9]:
from typing import Optional


def propose_move(
        elf: Coord,
        elves: Set[Coord],
        preferences: List[Tuple[Tuple[str, ...], Coord]]) -> Optional[Coord]:
    ns = neighbours(elf, elves)
    if not ns:
        return None
    for headings, move in preferences:
        if not any(heading in ns.keys() for heading in headings):
            return add_tuples(elf, move)
    return None

In [10]:
from collections import Counter


def agree_moves(elves: Set[Coord],
                proposed_moves: Dict[Coord, Coord]) -> Dict[Coord, Coord]:
    counter = Counter(proposed_moves.values())
    output: Dict[Coord, Coord] = dict()
    for elf, proposed_move in proposed_moves.items():
        if counter[proposed_move] == 1:
            output[elf] = proposed_move
    return output

In [11]:
def do_moves(elves: Set[Coord], moves: Dict[Coord, Coord]) -> Set[Coord]:
    output = elves.copy()
    for move_from, move_to in moves.items():
        output.remove(move_from)
        output.add(move_to)
    return output

In [12]:
def do_round(elves: Set[Coord], round: int) -> Tuple[Set[Coord], bool]:
    proposed_moves: Dict[Coord, Coord] = dict()
    rotation = (round - 1) % 4
    preferences = PREFERENCES[rotation:] + PREFERENCES[:rotation]
    for elf in elves:
        if proposed_move := propose_move(elf, elves, preferences):
            proposed_moves[elf] = proposed_move
    moves: Dict[Coord, Coord] = agree_moves(elves, proposed_moves)
    if len(moves.keys()) == 0:
        return elves, False
    else:
        return do_moves(elves, moves), True

In [13]:
elves = parse_data(testdata)
for round in range(1, 11):
    elves, _ = do_round(elves, round)
    display(elves_str(elves))
assert count_spaces(elves) == 110

['..........',
 '......#...',
 '....#...#.',
 '..#..#.#..',
 '......#..#',
 '...#.#.##.',
 '.#..#.#...',
 '.#.#.#.##.',
 '..........',
 '...#..#...',
 '..........']

['............',
 '.......#....',
 '....#.....#.',
 '...#..#.#...',
 '.......#...#',
 '...#..#.#...',
 '.#...#.#.#..',
 '............',
 '..#.#.#.##..',
 '....#..#....',
 '............']

['............',
 '.......#....',
 '.....#....#.',
 '..#..#...#..',
 '.......#...#',
 '...#..#.#...',
 '.#..#.....#.',
 '.......##...',
 '..##.#....#.',
 '...#........',
 '.......#....',
 '............']

['............',
 '.......#....',
 '......#....#',
 '..#...##....',
 '...#.....#.#',
 '.........#..',
 '.#...###..#.',
 '..#......#..',
 '....##....#.',
 '....#.......',
 '.......#....',
 '............']

['............',
 '.......#....',
 '............',
 '..#..#.....#',
 '.........#..',
 '......##...#',
 '.#.#.####...',
 '...........#',
 '....##..#...',
 '..#.........',
 '..........#.',
 '....#..#....',
 '............']

['............',
 '.......#....',
 '............',
 '..#..#.....#',
 '......##.#..',
 '...........#',
 '.#.#........',
 '.....####..#',
 '........#...',
 '..#.##......',
 '..........#.',
 '....#..#....',
 '............']

['............',
 '.......#....',
 '............',
 '..#.#......#',
 '........##..',
 '......#....#',
 '.#.#..##....',
 '....#....#.#',
 '.........#..',
 '..##..#.....',
 '..........#.',
 '....#..#....',
 '............']

['............',
 '.......#....',
 '............',
 '..#.#...#..#',
 '......#...#.',
 '...#.......#',
 '.#......#...',
 '.....##...##',
 '..#.......#.',
 '....#.#.....',
 '..........#.',
 '....#..#....',
 '............']

['............',
 '.......#....',
 '...........#',
 '..#.#...#...',
 '......#..#..',
 '...#.......#',
 '.#...##.#.#.',
 '...........#',
 '..#.........',
 '....#.#...#.',
 '..........#.',
 '....#..#....',
 '............']

['.............',
 '.......#.....',
 '...........#.',
 '..#.#..#.....',
 '......#......',
 '...#.....#..#',
 '.#......##...',
 '.....##......',
 '..#........#.',
 '....#.#..#...',
 '.............',
 '....#..#..#..',
 '.............']

In [14]:
elves = parse_data(inputdata)
for round in range(1, 11):
    elves, _ = do_round(elves, round)
count_spaces(elves)

3684

In [15]:
HTML(downloaded['part1_footer'])

## Part Two

In [16]:
HTML(downloaded['part2'])

In [17]:
from itertools import count

elves = parse_data(testdata)
for round in count(start=1):
    elves, moving = do_round(elves, round)
    if not moving:
        break
assert round == 20

In [18]:
elves = parse_data(inputdata)
for round in count(start=1):
    elves, moving = do_round(elves, round)
    if not moving:
        break
display(round)

862

In [19]:
HTML(downloaded['part2_footer'])